# Deploy open-source Large Language Models on Amazon SageMaker

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

---

In this notebook, we will show you how to deploy the open-source LLMs from HuggingFace on Amazon SageMaker. The notebook contains three sections:
- Section 1: Deploy Falcon model and embedding model to Amazon SageMaker
- Section 2: Use RAG based approach with [LangChain](https://python.langchain.com/en/latest/index.html) and SageMaker endpoints to build a simplified question and answering application.

***
This notebook is designed to run on `Python 3 Data Science 3.0` kernel in Amazon SageMaker Studio
***

#### 1. Setup development environment

We are going to use the `sagemaker` python SDK to deploy BLOOM to Amazon SageMaker. We need to make sure to have an AWS account configured and the `sagemaker` python SDK installed. You can safely ignore the errors from the pip install if there is any. 

In [ ]:
!pip install sagemaker==2.180.0 boto3==1.28.35 --quiet
!pip install ipywidgets==7.0.0 langchain==0.0.148 faiss-cpu==1.7.4 unstructured==0.10.6 --quiet --use-deprecated=legacy-resolver

## Secton 1: Deploy Falcon model and embedding model to Amazon SageMaker
In this section, we will deploy the open-source [Falcon 7b instruct model](https://huggingface.co/tiiuae/falcon-7b-instruct) on SageMaker for real-time inference. 
To deploy [Falcon-7B-Instruct](https://huggingface.co/tiiuae/falcon-7b-instruct) to Amazon SageMaker we create a `HuggingFaceModel` model class and define our endpoint configuration including the `hf_model_id`, `instance_type` etc. We will use a `g5.2xlarge` instance type.


This is an example on how to deploy the open-source LLMs to Amazon SageMaker for inference using the [Large Model Inference (LMI)](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-dlc.html) container from DLC to SageMaker and run inference with it. We will deploy the 7B-Instruct [Falcon](https://huggingface.co/tiiuae/falcon-7b-instruct) an open-source Chat LLM trained by TII.



In [ ]:
import sagemaker
import boto3
import json
import time
from sagemaker import Model, image_uris, serializers, deserializers

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

bucket = sess.default_bucket()
default_bucket_prefix = sess.default_bucket_prefix
region = boto3.Session().region_name
print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {region}")
sm_client = boto3.client("sagemaker")
smr = boto3.client("sagemaker-runtime")

### Start preparing model artifacts
In LMI container, we expect some artifacts to help setting up the model
- serving.properties (required): Defines the model server settings
- model.py (optional): A python file to define the core inference logic
- requirements.txt (optional): Any additional pip wheel need to install

In the **serving.properties** file defines the engine to use and model to host. Note the `tensor_parallel_degree` parameter which is also required in this scenario. We will use tensor parallelism to divide the model into multiple parts because no single GPU has enough memory for the entire model. In this case we will use a 'ml.g5.2xlarge' instance which provides 1 GPU. Be careful not to specify a value larger than the instance provides or your deployment will fail.

In [ ]:
%%writefile serving.properties
engine=MPI
option.model_id=tiiuae/falcon-7b-instruct
option.trust_remote_code=true
option.tensor_parallel_degree=1
option.paged_attention=true
option.max_rolling_batch_size=64
option.rolling_batch=lmi-dist
option.max_rolling_batch_prefill_tokens=1560


In [ ]:
%%sh
mkdir mymodel-7b
mv serving.properties mymodel-7b/
tar czvf mymodel-7b.tar.gz mymodel-7b/
rm -rf mymodel-7b

### Start building SageMaker endpoint
In this step, we will build SageMaker endpoint from scratch

#### Getting the container image URI

In [ ]:
image_uri = image_uris.retrieve(framework="djl-deepspeed", region=region, version="0.23.0")
image_uri

Then we upload the artifacts on S3 and create SageMaker model

In [ ]:
s3_code_prefix = "large-model-lmi/code"

# If a default bucket prefix is specified, append it to the default bucket
if default_bucket_prefix:
    s3_code_prefix = f"{default_bucket_prefix}/{s3_code_prefix}"

code_artifact = sess.upload_data("mymodel-7b.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")
falcon_model_name = sagemaker.utils.name_from_base("lmi-model-falcon-7b")
model = Model(
    sagemaker_session=sess,
    image_uri=image_uri,
    model_data=code_artifact,
    role=role,
    name=falcon_model_name,
)

#### Create SageMaker endpoint

We now can call the deploy function to create the LLM endpoint. You need to specify the instance to use and endpoint names.

In [ ]:
instance_type = "ml.g5.2xlarge"
endpoint_name = falcon_model_name

model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=900,
    wait=False,
)

SageMaker will now create our endpoint and deploy the model to it. This can take 10-15 minutes. During this time, please continue the following section to deploy the embedding model for the RAG solution. We will invoke the deployed endpoint when all the models are deployed successfully.

To see more model deployment examples, you can find an example notebook [here at the SageMaker examples gitrepo.](https://github.com/aws/amazon-sagemaker-examples/tree/51cbf4a77c98dc9b74fde6a8c47db0dad40fb910/inference/generativeai)

#### Deploy the GPT-J 6B embedding on SageMaker using SageMaker Jumpstart

In this section, we host the pre-trained [GPT-J-6B](https://huggingface.co/EleutherAI/gpt-j-6b) Hugging Face sentence transformer model, into SageMaker and generate an embedding vector with 4096 dimensions of the input text string. In this lab, we will use the `GPT-J 6B Embedding FP16` provided by SageMaker Jumpstart which loads a 16-bit quantized version of the original model by specifying the half-precision dtype, torch.float16. By using half precision, this model consumes less GPU memory and performs faster inference than the full precision version. For more information, please view the Hugging Face documentation for [FP16 optimization](https://huggingface.co/docs/diffusers/optimization/fp16).

There are different ways you can choose to deploy the GPT-J-6B model. Here we show you two options:
- deploy the GPT-J-6B embedding model from the Jumpstart UI
- deploy the GPT-J-6B embedding model using SageMaker python SDK

Please choose only one of the below two options to deploy the embedding model.

#### Option 1: Deploy the GPT-J-6B embedding model from the Jumpstart UI
On the left-hand-side navigation pane, got to **Home**, under **SageMaker JumpStart**, choose **Model, notebooks, solutions**. You’re presented with a range of solutions, foundation models, and other artifacts that can help you get started with a specific model or a specific business problem or use case. If you want to experiment in a particular area, you can use the search function. Or you can simply browse the artifacts to find the relevant model or business solution for your needs. To start exploring the Stable Diffusion models, complete the following steps:

1. Go to the **Foundation Models** section. In the search bar, search for the **embedding** model and select the **GPT-J 6B Embedding FP16**.

![Image jumpstart](./img/embedding_model.png)

2. A new tab is opened with the options to train, deploy and view model details as shown below. In the Deploy Model section, expand Deployment Configuration. For SageMaker hosting instance, choose the hosting instance (for this lab, we use ml.g5.4xlarge). You can also change the Endpoint name as needed. Then click the Deploy button.

![Image deploy](./img/embedding_deploy.png)

3. The deploy action will start a new tab showing the model creation status and the model deployment status. 

While the endpoint is deploying, update the embedding endpoint name in the following cell.

In [ ]:
endpoint_name_embed = "<your embedding model endpoint name>"  # change the endpoint name

Now you can directly go to section **[Wait until all the endpoints are up and running](#Wait-until-all-the-endpoints-are-up-and-running)**.

#### Option 2: deploy the GPT-J-6B embedding model using SageMaker python SDK
Now we will show you how to use code to deploy the pretrained models from SageMaker Jumpstart using the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker).

In [ ]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor
import os

instance_type = "ml.g5.4xlarge"  # instance type to use for deployment
model_version = "*"
env = {"SAGEMAKER_MODEL_SERVER_WORKERS": "1", "TS_DEFAULT_WORKERS_PER_MODEL": "1"}

We can directly load the pretrained model artifacts from SageMaker JumpStart. The SageMaker Python SDK uses model IDs and model versions to access the necessary utilities for pre-trained models. The [table provided by the readme doc](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html) serves to provide the core material plus some extra information that can be useful in selecting the correct model ID and corresponding parameters.

In [ ]:
model_id = "huggingface-textembedding-gpt-j-6b-fp16"

model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)
embed_endpoint_name = sagemaker.utils.name_from_base(model_id)

# Retrieve the inference container uri. This is the base HuggingFace container image for the default model above.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=instance_type,
)
model_inference = Model(
    image_uri=deploy_image_uri,
    model_data=model_uri,
    role=role,
    predictor_cls=Predictor,
    name=model_id,
    env=env,
)
model_predictor_inference = model_inference.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    predictor_cls=Predictor,
    endpoint_name=embed_endpoint_name,
    wait=False,
)
print(f"Model {model_id} has been created successfully.")

### Wait until all the endpoints are up and running

In [ ]:
# wait for the endpoint to be deployed successfully
def wait_for_endpoint(endpoint_name=None):
    describe_endpoint_response = sm_client.describe_endpoint(EndpointName=endpoint_name)

    while describe_endpoint_response["EndpointStatus"] == "Creating":
        describe_endpoint_response = sm_client.describe_endpoint(EndpointName=endpoint_name)
        print(describe_endpoint_response["EndpointStatus"])
        time.sleep(15)

    print(f"endpoint {endpoint_name} is in service now.")
    return

In [ ]:
for ep_name in [endpoint_name, embed_endpoint_name]:
    wait_for_endpoint(ep_name)

### Test endpoint outputs
Now we can invoke each endpoint to test the endpoint outputs. First, let's check the text-to-text endpoint using Falcon model.

In [ ]:
body = {"inputs": "what is life?", "parameters": {"max_new_tokens": 400, "return_full_text": False}}
output = smr.invoke_endpoint(
    EndpointName=endpoint_name, Body=json.dumps(body), ContentType="application/json"
)
resp = json.loads(output["Body"].read().decode("utf8"))
print(resp["generated_text"])

Then run the follow code to generate embeddings of the input using the embedding model.

In [ ]:
input_str = {"text_inputs": resp["generated_text"]}
output = smr.invoke_endpoint(
    EndpointName=embed_endpoint_name, Body=json.dumps(input_str), ContentType="application/json"
)
embeddings = output["Body"].read().decode("utf-8")
print(embeddings)

## Section2: Use RAG based approach with [LangChain](https://python.langchain.com/en/latest/index.html) and SageMaker endpoints to build a simplified question and answering application.


We plan to use document embeddings to fetch the most relevant documents in our document knowledge library and combine them with the prompt that we provide to LLM.

To achieve that, we will do following.

1. **Generate embedings for each of document in the knowledge library with Huggingface all-MiniLM-L6-v2 embedding model.**
2. **Identify top K most relevant documents based on user query.**
    - 2.1 **For a query of your interest, generate the embedding of the query using the same embedding model.**
    - 2.2 **Search the indexes of top K most relevant documents in the embedding space using in-memory Faiss search.**
    - 2.3 **Use the indexes to retrieve the corresponded documents.**
3. **Combine the retrieved documents with prompt and question and send them into SageMaker LLM.**



Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt -- maximum sequence length. 

---
To build a simplied QA application with LangChain, we need: 
1. Wrap up our SageMaker endpoints for embedding model and LLM into `langchain.embeddings.SagemakerEndpointEmbeddings` and `langchain.llms.sagemaker_endpoint.SagemakerEndpoint`. That requires a small overwritten of `SagemakerEndpointEmbeddings` class to make it compatible with SageMaker embedding mdoel.
2. Prepare the dataset to build the knowledge data base. 

---

Wrap up our SageMaker endpoints for embedding model into `langchain.embeddings.SagemakerEndpointEmbeddings`. That requires a small overwritten of `SagemakerEndpointEmbeddings` class to make it compatible with SageMaker embedding model.

In [ ]:
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from typing import Any, Dict, List, Optional
import json


class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(self, texts: List[str], chunk_size: int = 5) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size
        for i in range(0, len(texts), _chunk_size):
            response = self._embedding_func(texts[i : i + _chunk_size])
            print
            results.extend(response)
        return results


class ContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        embeddings = response_json["embedding"]
        return embeddings


content_handler = ContentHandler()

embeddings = SagemakerEndpointEmbeddingsJumpStart(
    endpoint_name=embed_endpoint_name,
    region_name=region,
    content_handler=content_handler,
)

Next, we wrap up our SageMaker endpoints for LLM into `langchain.llms.sagemaker_endpoint.SagemakerEndpoint`. 

In [ ]:
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint

parameters = {"max_new_tokens": 500, "return_full_text": False, "temperature": 0.1}


class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        input_str = json.dumps({"inputs": prompt, "parameters": {**model_kwargs}})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = output.read()
        res = json.loads(response_json)
        ans = res["generated_text"]
        return ans


content_handler = ContentHandler()

sm_llm = SagemakerEndpoint(
    endpoint_name=endpoint_name,
    region_name=region,
    model_kwargs=parameters,
    content_handler=content_handler,
)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader

Use langchain to read the `txt` data. There are multiple built-in functions in LangChain to read different format of files such as `csv`, `html`, and `pdf`. For details, see [LangChain document loaders](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html).

In [ ]:
loader = DirectoryLoader("./data/", glob="**/*.txt")
documents = loader.load()

We generate embedings for each of document in the knowledge library with Huggingface all-MiniLM-L6-v2 embedding model.documents

In [ ]:
docsearch = FAISS.from_documents(documents, embeddings)

In [ ]:
question = "what is the recommended way to first customize a foundation model?"

Based on the question above, we then **identify top K most relevant documents based on user query, where K = 3 in this setup**.

In [ ]:
docs = docsearch.similarity_search_with_score(question)

In [ ]:
docs

In [ ]:
source = []
context = []
for doc, score in docs:
    context.append(doc)
    source.append(doc.metadata["source"].split("/")[-1])

Finally, we **combine the retrieved documents with prompt and question and send them into SageMaker LLM.** 

We define a customized prompt as below.

In [ ]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.:\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [ ]:
chain = load_qa_chain(llm=sm_llm, prompt=PROMPT)

In [ ]:
result = chain({"input_documents": context, "question": question}, return_only_outputs=True)[
    "output_text"
]
print(result)

### Run the Question and Answering chatbot application

Once all the endpoints are deployed successfully, you can open a terminal in SageMaker Studio and use the below command to run the chatbot [Streamlit](https://streamlit.io/) application. Note that you need to install the required python packages that are specified in the “requirements.txt” file. You also need to update the environment variables with the endpoint names deployed in your account accordingly. When you execute the `chatbot-steamlit.py` file, it will automatically update the endpoint names based on the environment variables.

```
$ pip install -r requirements.txt
$ export nlp_ep_name=<the falcon endpoint name deployed in your account>
$ export embed_ep_name=<the embedding endpoint name deployed in your account>
$ streamlit run chatbot-streamlit.py --server.port 6006 --server.maxUploadSize 6
```

To access the Streamlit UI, copy your SageMaker Studio url and replace `lab?` with `proxy/[PORT NUMBER]/`. Because we specified the server port to 6006, so the url should look like:

```
https://<domain ID>.studio.<region>.sagemaker.aws/jupyter/default/proxy/6006/
```

Replace the domain ID and region with the correct value in your account to access the UI as below:
![streamlitUI](./img/Streamlit_UI.png)

You can find some suggested `prompt` on the left-hand-side sidebar. When you upload the sample files (you can find the sample files in the test folder), the chatbot will automatically provide prompt suggestions based on the input data type.

**Congratulations on finishing lab 1 !!!**

_____________________________________________________________________________

## clean up

In [ ]:
## The below lines will delete the falcon model endpoint, change the endpoint and model name to delete other resources created.
# sm_client.delete_endpoint(EndpointName=endpoint_name)
# sm_client.delete_model(ModelName=falcon_model_name)
# sm_client.delete_endpoint(ModelName=embed_endpoint_name)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/inference|generativeai|llm-workshop|chatbot-apps|build_cahtbot_applications_using_rag_on_sagemaker.ipynb)

